In [15]:
import numpy as np
import json

def openLists(route):
    val = {}
    with open(f'{route}') as file:
        for line in file:
            data = json.loads(line)
            key, values = data
            val[key] = np.array(values)
        return val
    

archivo = openLists('listas-posteo-100.json')

In [16]:
import random
import itertools
listas = list(archivo.values())
# Generamos todas las posibles combinaciones de 3 listas
def generar_combinaciones(listas, n):
    return itertools.combinations(listas, n)

# Seleccionamos aleatoriamente 1000 combinaciones
combinaciones = generar_combinaciones(listas, 2)
A = random.sample(list(combinaciones), 1000)
combinaciones = generar_combinaciones(listas, 3)
B = random.sample(list(combinaciones), 1000)
combinaciones = generar_combinaciones(listas, 4)
C = random.sample(list(combinaciones), 1000)

#print(B)  # Imprime las 1000 combinaciones seleccionadas

In [17]:
def adaptive_binary_search(arr, x, last_found_index=0):
  low = max(0, last_found_index - 1)  # Iniciar la búsqueda desde el último elemento encontrado o el inicio del arreglo
  high = len(arr) - 1

  while low <= high:
    mid = (low + high) // 2

    if arr[mid] == x:
      return mid
    elif arr[mid] < x:
      low = mid + 1
    else:
      high = mid - 1

  return -1

In [18]:
def total_binary_search(arr, x):
  low = 0
  high = len(arr) - 1

  while low <= high:
    mid = (low + high) // 2
    if arr[mid] == x:
      return mid
    elif arr[mid] < x:
      low = mid + 1
    else:
      high = mid - 1

  return -1

In [19]:
def rounded_binary_search(arr, x, last_found_index=0):
  low = max(0, last_found_index - 1)  # Iniciar la búsqueda desde el último elemento encontrado o el inicio del arreglo
  high = len(arr) - 1

  while low <= high:
    mid = (low + high) // 2

    if arr[mid] == x:
      return mid
    elif arr[mid] < x:
      # Si el elemento buscado es mayor, ajustar el límite inferior
      # Considerar ajustar el límite inferior más hacia la derecha si las comparaciones anteriores
      # indican que el elemento está más cerca de la parte superior del arreglo
      low = mid + 1
    else:
      # Si el elemento buscado es menor, ajustar el límite superior
      # Considerar ajustar el límite superior más hacia la izquierda si las comparaciones anteriores
      # indican que el elemento está más cerca de la parte inferior del arreglo
      high = mid - 1

  return -1

In [31]:
import time

def SvS(sets, k, fbusq):
  start_time = time.process_time()
  comparisons = 0

  # 1. Ordenar los conjuntos por tamaño
  sets.sort(key=len)

  # 2. El conjunto más pequeño es el candidato inicial
  candidate_set = sets[0].copy()

  # 3. Inicializar índices
  indices = [0] * k

  # 4. Iterar sobre los conjuntos
  for i, S in enumerate(sets):
    for e in list(candidate_set):
      # 6. Búsqueda binaria redondeada
      index = fbusq(S[indices[i]:], e)
      comparisons += 1
      if index != -1:
        indices[i] += index + 1
      else:
        # 8-10. Eliminar e del conjunto candidato
        candidate_set.remove(e)
  end_time = time.process_time()
  elapsed_time = end_time - start_time


  return candidate_set, comparisons, elapsed_time

# Ejemplo de uso:
sets = [[1, 2, 3, 4], [3, 4, 9, 10], [3, 4, 5, 6, 7], [3, 4, 11, 12]]
k = len(sets)
result, comparaciones, tiempo = SvS(sets, k, total_binary_search)
print(result)  # Output: {3, 4}
print(comparaciones)
print(tiempo)

[3, 4]
12
6.199999999978445e-05


In [41]:
flist = []
ftime = []
fcomp = []
for element in C:
    lista1 = element[0].tolist()
    lista2 = element[1].tolist()
    lista3 = element[2].tolist()
    lista4 = element[3].tolist()
    listafinal = [lista1, lista2, lista3, lista4]
    k = len(listafinal)
    result, comparaciones, tiempo = SvS(listafinal, k, total_binary_search)
    flist.append(result)
    ftime.append(tiempo)
    fcomp.append(comparaciones)

print(flist)
    

[[], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [8364, 27166], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [22730], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [5245, 15019, 17414, 17677, 27551, 28680], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [4110], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [260, 262, 272, 338, 6659, 10701], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [

In [40]:
result = [sublist for sublist in flist if sublist]
print(result)

[[8364, 27166], [22730], [5245, 15019, 17414, 17677, 27551, 28680], [4110], [260, 262, 272, 338, 6659, 10701], [16690], [13355], [23327, 23572, 40195], [19689, 22388], [22145], [45453], [12723], [29098], [18359, 21534], [6996, 7060], [6640], [19518], [7219], [13706, 15631], [1279], [23015, 30176], [10487, 18586], [38117, 48087], [40533], [6013, 6250, 6399, 9938, 11652, 17425, 42904, 42910], [16650, 16651, 16652, 16661, 16710, 16712, 16963, 17244, 17279, 17347, 17843, 17919, 24504, 43470], [40631], [7604, 8043, 8118, 8160, 8236, 9820, 31516, 31626, 44260, 44269, 44296, 44326, 44389, 44732, 44780, 44788, 44793, 44862, 44914, 45093, 45181, 45280, 45286, 45444, 46160, 46213], [3785, 3954], [14990, 16265, 22927, 37680], [25914, 26884, 29357], [45366], [45679], [2925], [16163], [11092], [17575, 31327, 32652, 37616], [28100, 36043, 46988], [19518, 22402, 31353, 31373, 34933, 35292, 35311, 45370], [3300, 3778, 26884], [41059], [45345], [40499], [46610], [3513], [5145, 5694, 6452, 7041, 7105, 7

In [46]:
def starter(listas, busqueda, interseccion, combs):
    flist = []
    ftime = []
    fcomp = []
    for element in listas:
        if combs == 2:
            lista1 = element[0].tolist()
            lista2 = element[1].tolist()
            listafinal = [lista1, lista2,]
        if combs == 3:
            lista1 = element[0].tolist()
            lista2 = element[1].tolist()
            lista3 = element[2].tolist()
            listafinal = [lista1, lista2, lista3]
        if combs == 4:
            lista1 = element[0].tolist()
            lista2 = element[1].tolist()
            lista3 = element[2].tolist()
            lista4 = element[3].tolist()
            listafinal = [lista1, lista2, lista3, lista4]
        k = len(listafinal)
        result, comparaciones, tiempo = interseccion(listafinal, k, busqueda)
        flist.append(result)
        flist = [sublist for sublist in flist if sublist] #Eliminamos las listas vacías
        ftime.append(tiempo)
        fcomp.append(comparaciones)
        64808
    return flist, ftime, fcomp
test21, test22, test23 = starter(A, adaptive_binary_search, SvS, 2)

In [49]:
print(test11)
print(test21)

[[3441, 3785, 4499, 4678, 4696, 4754, 5183, 6300, 7295, 7331, 7332, 7361, 7404, 8420, 9218, 9336, 9753, 9932, 10348, 10425, 10838, 10850, 11225, 13415, 13640, 13660, 13838, 14010, 14110, 14418, 14426, 14557, 16049, 16672, 17814, 20149, 20165, 24602, 26529, 26583, 30606, 36167, 36776, 37311, 38448, 38801, 38961, 39008, 39011, 39142, 40089, 40156, 40375, 42310, 42765, 43402, 43489, 43509, 43575, 43645, 43784, 43914, 43974, 44392, 44978, 45177, 45264, 47724, 48228], [38489, 39256, 39674], [6390, 8733, 27145, 27269, 27284, 27356, 27569, 27826, 27876, 27884, 27954, 27995, 28940, 29207, 29244, 29473, 29532, 29664, 29667, 29731, 30091, 30144, 32710, 34250, 49918], [3168, 5306, 5749, 19935, 20312, 21173, 21748, 22249, 24220, 24259, 24485, 26792, 27092, 29882, 30206, 32338, 35432, 39140, 39507, 42313, 44054, 45248, 46517, 46781, 47335, 48774], [28927, 29642], [671, 1038, 3072, 4209, 4213, 4223, 6052, 7456, 8212, 10487, 10583, 11011, 11543, 11694, 13310, 18586, 18691, 21994, 24533, 26257, 27484,

In [28]:
print(ftime)

[0.008597999999999217, 0.003906999999998106, 0.008822000000002106, 0.003983000000005177, 0.006633999999998252, 0.007137000000000171, 0.006332000000000448, 0.0029359999999982733, 0.02174699999999774, 0.003489999999999327, 0.0023249999999990223, 0.002691999999996142, 0.0026779999999959614, 0.0035160000000047376, 0.003157999999999106, 0.006686999999999443, 0.005507999999998958, 0.004022999999996557, 0.0037370000000009895, 0.002949999999998454, 0.006064999999999543, 0.002921999999998093, 0.011009000000001379, 0.003969999999995366, 0.00315599999999705, 0.003368000000001814, 0.002879000000000076, 0.0035189999999971633, 0.00430700000000428, 0.008052999999996757, 0.0044770000000013965, 0.0103160000000031, 0.0029399999999952797, 0.14158599999999666, 0.0027349999999941588, 0.003495000000000914, 0.002682000000000073, 0.002833000000002528, 0.0037289999999998713, 0.003555000000005748, 0.004702999999999236, 0.004225000000005252, 0.0052329999999969345, 0.004168999999997425, 0.003487999999997271, 0.00